In [1]:
import numpy as np
import pandas as pd
import webhoseio
from aylienapiclient import textapi
import re
from urllib.parse import urlparse

In [2]:
# Ashwin's key - we only get 1000 free requests a month, so be cautious with the number of times you perform queries
webhoseio.config(token="456af32b-3c58-455d-b9f8-90875bfc8f58") 

In [3]:
client = textapi.Client("4657dfa6", "bbfd5faa23a1020d92e946dd83bce6b7")

In [21]:
webhoseio.query?

In [76]:
output_ex = webhoseio.query("filterWebData", {"q":u"thread.title:(Vox Claims North Korea's Economy Doing 'Pretty Well') site:breitbart.com"})

In [77]:
output_ex

{'moreResultsAvailable': 1,
 'next': '/search?token=456af32b-3c58-455d-b9f8-90875bfc8f58&format=json&ts=1493779148000&q=thread.title%3A%28Vox+Claims+North+Korea%27s+Economy+Doing+%27Pretty+Well%27%29+site%3Abreitbart.com',
 'posts': [{'author': 'John Hayward',
   'crawled': '2017-05-03T04:29:26.000+03:00',
   'entities': {'locations': [{'name': 'north korea', 'sentiment': 'none'},
     {'name': 'pyongyang', 'sentiment': 'none'}],
    'organizations': [{'name': 'vox', 'sentiment': 'negative'}],
    'persons': [{'name': 'ed jones', 'sentiment': 'none'},
     {'name': 'kim il-sung', 'sentiment': 'none'},
     {'name': 'kim jong-un', 'sentiment': 'none'}]},
   'external_links': [],
   'highlightText': '',
   'highlightTitle': '',
   'language': 'english',
   'ord_in_thread': 0,
   'published': '2017-05-03T04:29:26.000+03:00',
   'rating': None,
   'text': 'Vox Claims North Korea’s Economy Doing ‘Pretty Well’ ED JONES/AFP/Getty Images by John Hayward 2 May 2017 0 \n2 May, 2017 2 May,  \n“Tu

## Get the title and site_url of an article given a url

In [75]:
def getTitleAndSiteUrl(url):
    extract = client.Extract({"url": url, "best_image": False})
    title = extract['title']
    site_url = re.sub('www.', '', urlparse(url).hostname)
    return title, site_url

getTitleAndSiteUrl('http://www.breitbart.com/national-security/2017/05/02/north-korea-economy-not-doing-pretty-well/')

("Vox Claims North Korea's Economy Doing 'Pretty Well'", 'breitbart.com')

## Create the dataframe with the 20 features that are in the Kaggle Data Set. Also create the function that will add rows to the dataframe

In [16]:
# all the columns from the Kaggle data set
cols = ["uuid", "ord_in_thread", "author", "published", "title", "text", "language", "crawled", "site_url", "country", "domain_rank", "thread_title", "spam_score", "main_img_url", "replies_count", "participants_count", "likes", "comments", "shares", "type"]
df = pd.DataFrame(columns = cols)
df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type


In [63]:
def addRow(df, site_url, title):
    output = webhoseio.query("filterWebData", {"q":"thread.title:(" + title + ") site:" + site_url})
    post = output["posts"][0]
    print(output)
    thread = post["thread"]
    social = thread["social"]["facebook"]
    
    uuid = post["uuid"]
    ord_in_thread = post["ord_in_thread"]
    author = post["author"]
    published = post["published"]
    title = post["title"]
    text = post["text"]
    language = post["language"]
    crawled = post["crawled"]
    site_url = thread["site"]
    country = thread["country"]
    domain_rank = thread["domain_rank"]
    thread_title = thread["title"]
    spam_score = thread["spam_score"]
    img = thread["main_image"]
    replies = thread["replies_count"]
    participants = thread["participants_count"]
    likes = social["likes"]
    comments = social["comments"]
    shares = social["shares"]
    
    #data = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]]
    data = [[uuid, ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, img, replies, participants, likes, comments, shares, "N/A"]]
    df_row = pd.DataFrame(data, columns=cols)
    #print(df_row)
    df = df.append(df_row, ignore_index=True)
    return df

In [78]:
title, site_url = getTitleAndSiteUrl('http://www.breitbart.com/national-security/2017/05/02/north-korea-economy-not-doing-pretty-well/')
print(title)
print(site_url)

df = addRow(df, site_url, title)
df

Vox Claims North Korea's Economy Doing 'Pretty Well'
breitbart.com
{'posts': [{'highlightTitle': '', 'thread': {'site': 'breitbart.com', 'main_image': None, 'replies_count': 0, 'site_section': 'http://www.breitbart.com', 'site_categories': [], 'domain_rank': 967, 'site_full': 'www.breitbart.com', 'country': 'US', 'site_type': 'blogs', 'title': 'Vox Claims North Korea’s Economy Doing ‘Pretty Well’', 'published': '2017-05-03T04:29:26.000+03:00', 'title_full': 'Vox Claims North Korea’s Economy Doing ‘Pretty Well’', 'social': {'gplus': {'shares': 0}, 'vk': {'shares': 0}, 'linkedin': {'shares': 2}, 'stumbledupon': {'shares': 0}, 'facebook': {'likes': 25, 'comments': 0, 'shares': 25}, 'pinterest': {'shares': 2}}, 'performance_score': 0, 'participants_count': 1, 'uuid': '38f87e33973a01fbd5fca7b58254331a51402dd7', 'section_title': 'Breitbart News', 'url': 'http://omgili.com/ri/jHIAmI4hxg9bm_w_dC_byE8kjeeQbIOUDXhe_L3psUmT9okT02FFmsuJTsOp0uwxHhMo8o1qC03jNtlx2Bya6cXzY5lcYEQvRyM0QB7QRjW2TG3bGAcKcG

,author,comments,country,crawled,domain_rank,language,likes,main_img_url,ord_in_thread,participants_count,published,replies_count,shares,site_url,spam_score,text,thread_title,title,type,uuid
0,Josh Barro (jbarro@businessinsider.com),0.0,US,2017-05-03T03:42:43.000+03:00,283.0,english,0.0,None,0.0,1.0,2017-05-03T03:42:43.000+03:00,0.0,0.0,businessinsider.com,0.0,AP \nI've been reluctant to focus too much on ...,"Some thoughts on Greenwich, Winnetka, and why ...","Some thoughts on Greenwich, Winnetka, and why ...",NaN,0ef8b55a5f25489dc656fcea727cbfe2669ae93a
1,Josh Barro (jbarro@businessinsider.com),0.0,US,2017-05-03T03:42:43.000+03:00,283.0,english,0.0,None,0.0,1.0,2017-05-03T03:42:43.000+03:00,0.0,0.0,businessinsider.com,0.0,AP \nI've been reluctant to focus too much on ...,"Some thoughts on Greenwich, Winnetka, and why ...","Some thoughts on Greenwich, Winnetka, and why ...",N/A,0ef8b55a5f25489dc656fcea727cbfe2669ae93a
2,John Hayward,0.0,US,2017-05-03T04:29:26.000+03:00,967.0,english,25.0,None,0.0,1.0,2017-05-03T04:29:26.000+03:00,0.0,25.0,breitbart.com,0.0,Vox Claims North Korea’s Economy Doing ‘Pretty...,Vox Claims North Korea’s Economy Doing ‘Pretty...,Vox Claims North Korea’s Economy Doing ‘Pretty...,N/A,38f87e33973a01fbd5fca7b58254331a51402dd7


In [80]:
df = df[2:3]
df.to_pickle('bb_article.pkl')